In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import sqlite3
from scipy import sparse
import requests
import json
import os

# Data source

In [2]:
paper_table_file = "../data/testData.csv"

ES_PASSWORD = "FSailing4046"
ES_USERNAME = "skojaku"
ES_ENDPOINT = "iuni2.carbonate.uits.iu.edu:9200/wos/_search/" # wos/_search/

WOS_CITATION_DB = "../data/wos-citation.db"

In [10]:
paper_table = pd.read_csv(paper_table_file)
wos_ids = paper_table["UID"].drop_duplicates().values

# Retrieve Data from WOS database

In [40]:
# Set the end point for the ElasticSearch
es_end_point = "http://{user}:{password}@{endpoint}".format(
    user=ES_USERNAME, password=ES_PASSWORD, endpoint=ES_ENDPOINT
)

def find_papers_by_UID(uri, uids):
    """Simple Elasticsearch Query"""
    query = json.dumps({
        "query": {
            "ids": {
                "values": uids
            }
        },
        "size":len(uids),
    })
    headers = {"Content-Type": "application/json"}
    response = requests.get(uri, headers=headers, data=query)
    results = json.loads(response.text)
    return results

results = find_papers_by_UID(es_end_point, wos_ids.tolist())

# Create tables

In [41]:
import itertools

def safe_parse(parse_func):
    
    def wrapper(results, *args, **kwargs):
        df_list = []
        for result in results["hits"]["hits"]:
            UID = result["_id"]

            df = parse_func(result, *args, **kwargs)

            df["UID"] = UID
            df_list+=[df]
        df = pd.concat(df_list, ignore_index = True)
        return df
    return wrapper

@safe_parse
def parse_address_name(result):
    address_name = result["_source"]["doc"].get("address_name",[])
    merged = [r["address_spec"] for r in list(itertools.chain(*address_name))]
    df = pd.DataFrame(merged)
    return df
    
@safe_parse
def parse_author_name(result):
    author_name = result["_source"]["doc"].get("name",[])
    df = pd.DataFrame([r for r in list(itertools.chain(*author_name))])
    return df

@safe_parse
def parse_paper_info(result):
    #
    # Publication year
    #
    pub_info = result["_source"]["doc"].get("pub_info", [])
    if len(pub_info)>=1:
        pub_year = pub_info[0].get("_pubyear", float("NaN") )
    else:
        pub_year = float("NaN")

    #
    # Titles and source
    #
    titles = result["_source"]["doc"].get("titles", [])
    if len(titles)>0:
        titles = titles[0].get("title", [])
        title = ""
        source = ""
        source_iso = ""
        for r in titles: 
            if r["_type"] == "source":
                source = r["_VALUE"]
            elif r["_type"] == "abbrev_iso":
                source_iso = r["_VALUE"]
            elif r["_type"] == "item":
                title = r["_VALUE"]
    
    #
    # Grant number not implemented
    #
    grant_number = ""
    df = pd.DataFrame([{"source":source, "title":title, "source_iso":source_iso, "pub_year":pub_year, "grant_number":grant_number}])
    return df

address_table = parse_address_name(results)
author_table = parse_author_name(results)
paper_info = parse_paper_info(results)

## Make name_table and block_table

In [21]:
def get_initials(first_name, last_name):
    def get_first_char(x, default = ""):
        if isinstance(x, str):
            return x[0]
        else:
            return default
    return get_first_char(first_name) + get_first_char(last_name)

def get_normalized_name(first_name, last_name):
    def get_name(x, default = ""):
        if isinstance(x, str):
            return x.lower()
        else:
            return default
    def get_first_char(x, default = ""):
        if isinstance(x, str):
            return x[0]
        else:
            return default
        
    return get_name(last_name) + get_name(get_first_char(first_name)) 

name_table = author_table.copy()
name_table = name_table.rename(columns = {"full_name":"name"} )
    
name_table["initials"] = name_table.apply(lambda x : get_initials(x["first_name"], x["last_name"]), axis = 1)
name_table["normalized_name"] = name_table.apply(lambda x : get_normalized_name(x["first_name"], x["last_name"]), axis = 1)
name_table = name_table[["name", "initials", "first_name", "last_name", "normalized_name"]].drop_duplicates()

block_table = name_table[["normalized_name"]].drop_duplicates().reset_index().drop(columns = ["index"])
block_table["block_id"] = np.arange(block_table.shape[0])

name_table = pd.merge(name_table, block_table, on = "normalized_name", how = "left")
name_table["name_id"] = np.arange(name_table.shape[0])

## Make paper table

In [22]:
paper_table = paper_info.copy()
paper_table = paper_table.rename(columns = {"source":"journal", "UID":"paper_id"})

,journal,title,source_iso,pub_year,grant_number,paper_id
0,VIROLOGY,INDUCTION OF ANTIBODIES PROTECTING AGAINST TRA...,Virology,1995,,WOS:A1995TE73200023
1,IMMUNOLOGY,"Effect of adoptive transfer of CD4, CD8 and B ...",Immunology,1996,,WOS:A1996UP45900008
2,VIROLOGY,Coronavirus MHV-3-induced apoptosis in macroph...,Virology,1998,,WOS:000076662200005
3,PLOS ONE,A Metagenomic Analysis of Pandemic Influenza A...,PLoS One,2010,,WOS:000283045300008
4,HUMAN IMMUNOLOGY,CD209 (DC-SIGN)-336A > G promoter polymorphism...,Hum. Immunol.,2010,,WOS:000279493000009
...,...,...,...,...,...,...
304,JOURNAL OF VIROLOGICAL METHODS,Generation and Characterization of Eptesicus f...,J. Virol. Methods,2016,,WOS:000386190700027
305,SCIENCE TRANSLATIONAL MEDICINE,A synthetic consensus anti-spike protein DNA v...,Sci. Transl. Med.,2015,,WOS:000360941700006
306,JOURNAL OF VETERINARY INTERNAL MEDICINE,Respiratory Pathogens in Quebec Dairy Calves a...,J. Vet. Intern. Med.,2015,,WOS:000348664000054
307,EMERGING INFECTIOUS DISEASES,Feasibility of Using Convalescent Plasma Immun...,Emerg. Infect. Dis,2016,,WOS:000381955900004


## Make name_paper_table

In [30]:
name_paper_table = author_table.copy().rename(columns = {"full_name":"name", "UID":"paper_id", "email_addr":"email_address"})
name_paper_table = pd.merge(name_paper_table, name_table[["name_id", "name"]], on = "name", how = "left").drop(columns = "name")
name_paper_table = name_paper_table[["name_id", "paper_id", "email_address"]]

,name_id,paper_id,email_address
0,0,WOS:A1995TE73200023,NaN
1,1,WOS:A1995TE73200023,NaN
2,2,WOS:A1995TE73200023,NaN
3,3,WOS:A1995TE73200023,NaN
4,4,WOS:A1995TE73200023,NaN
...,...,...,...
2217,1586,WOS:000379829300001,NaN
2218,1587,WOS:000379829300001,NaN
2219,1594,WOS:000379829300001,NaN
2220,1589,WOS:000379829300001,NaN


## Make address table

In [42]:
address_table = address_table.rename(columns = {"organizations":"organization", "suborganizations":"department"})
address_table = address_table[["UID", "full_address", "city", "country", "organization", "department"]]
address_table["organization"] = address_table["organization"].astype(str)
address_table["department"] = address_table["department"].astype(str)

# Save to SQL db

In [43]:
OUTPUT_DB = "wos-disambiguation-data.db"
if os.path.exists(OUTPUT_DB):
    os.remove(OUTPUT_DB)
conn = sqlite3.connect(OUTPUT_DB)
cur = conn.cursor()

pd.DataFrame(name_paper_table).to_sql(
    "name_paper_table", conn, if_exists="append", index=False
)
pd.DataFrame(name_table).to_sql(
    "name_table", conn, if_exists="append", index=False
)
pd.DataFrame(block_table).to_sql(
    "block_table", conn, if_exists="append", index=False
)
pd.DataFrame(paper_table).to_sql(
    "paper_table", conn, if_exists="append", index=False
)

pd.DataFrame(address_table).to_sql(
    "address_table", conn, if_exists="append", index=False
)